In [ ]:
# Imports 
from IPython.display import Image, display
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import interact

import sys

# Add src folder
# TODO can be removed after a pip install
cwd = Path.cwd()
sys.path.insert(0, str(cwd.parents[0] / 'segwrap'))

# Import wrapper code for CellPose and check which device to use
import utils_cellpose

In [ ]:
# >>>>> Various widgets to define input parameters

# Textbox for path to be scanned
textbox_path_scan = widgets.Text(
    #placeholder='path-to-folder-with-data',
    value=r'D:\Documents\Data\test-data\cell-pose\jupyter-tests\example_data\analysis\segmentation-input',
    description='Data path:'
)

# Textbox for path to be scanned
textbox_path_save = widgets.Text(
    #placeholder='path-to-folder-to-save-data',
    value=r'D:\Documents\Data\test-data\cell-pose\jupyter-tests\example_data\analysis\segmentation-results',
    description='Results path:'
)

# Textbox to specify string for cytoplasmic images
textbox_str_cyto = widgets.Text(
    value='cy3',
    description='String cells:'
)

# Textbox to specify string for nuclei images
textbox_str_nuclei = widgets.Text(
    value='dapi',
    description='String nuclei:'
)


# Textbox to specify image extension
textbox_new_size = widgets.Text(
    value='512, 512',
    description='New size:',
)

# Textbox to specify image extension
textbox_img_ext = widgets.Text(
    value='.png',
    description='Image ext:',
)


# Slider to specify size of cells
slider_size_cells = widgets.IntSlider(
    value=100,
    min=30,
    max=500,
    step=5,
    description='Size cells:',
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


# Slider to specify size of nuclei
slider_size_nuclei=widgets.IntSlider(
    value=50,
    min=30,
    max=500,
    step=5,
    description='Size nuclei:',
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [ ]:
# >>>> Assemble widget for segmentation of cells and nuclei

# Button to perform prediction
button_predict_cells_nuclei = widgets.Button(
    description='Segment data',
    tooltip='Click to segment images of cells and nuclei',
    icon='angle-double-right'
)


# Define predict function
def predict_cells_nuclei(b=None):
    path_scan = textbox_path_scan.value
    path_save = textbox_path_save.value
    str_cyto = textbox_str_cyto.value
    str_nuclei = textbox_str_nuclei.value
    img_ext = textbox_img_ext.value
    new_size_str = textbox_new_size.value
    size_cells = slider_size_cells.value
    size_nuclei = slider_size_nuclei.value

    # Path containing the data
    if len(path_scan) == 0:
        print('Path containing data to be segmented has to be specified.')
        return
    
    path_scan = Path(path_scan)
    
    if not path_scan.is_dir():
        print('Path containing data to be segmented does not exist.')    
        return
    
    # Path to save data
    if len(path_save) == 0:
        print('Path containing data to be segmented has to be specified.')
        return
    
    path_save = Path(path_save)
    if not path_save.is_dir():
        path_save.mkdir(parents=True)

    
    # Check new size
    if len(new_size_str) == 0:
        new_size = ()
    else:
        if new_size_str.count(',') == 1:
            new_size = tuple(int(x) for x in new_size_str.split(','))
        else:
            print('String to define new size has to contain 1 \",\" to separate the new dimensions in XY.')
            return
            
    
    # Perform segmentation
    print('Performing segmentation ... for more details see tab \'Log\'.')
        
    # Call segmentation routine
    with tab_out:
        utils_cellpose.segment_cells_nuclei(path_scan=path_scan, 
                                      strings=(str_cyto, str_nuclei), 
                                      img_ext=img_ext, 
                                      new_size=new_size,
                                      sizes=(size_cells, size_nuclei), 
                                      models=('cyto','nuclei'),
                                      path_save=path_save)
    
    print('Segmentation finished')
    
    #  Show results
    print('\nSegmentation of cells:')
    @interact
    def show_images(file=path_save.glob('segmentation*cells*.png')):
        if file: 
            display(Image(file))
        
    # Print segmentation finished
    print('Segmentation of nuclei:')
    @interact
    def show_images(file=path_save.glob('segmentation*nuclei*.png')):
        if file:
            display(Image(file))
  
    
# Assign to button
@button_predict_cells_nuclei.on_click
def plot_on_click(b):
    predict_cells_nuclei()
    
    
# Arrange controls
tab_predict_cells_nuclei = widgets.VBox(children=[
                          textbox_path_scan,
                          textbox_path_save,    
                          textbox_str_cyto,
                          textbox_str_nuclei,
                          textbox_new_size,
                          textbox_img_ext,
                          slider_size_cells,
                          slider_size_nuclei,
                          button_predict_cells_nuclei

                      ])

# Tab for log
tab_out = widgets.Output(layout={'border': '1px solid black'})

# Entire interface
ui_segment_cells_nuclei = widgets.Tab(children=[tab_predict_cells_nuclei, tab_out])
ui_segment_cells_nuclei.set_title(0, 'Predict')
ui_segment_cells_nuclei.set_title(1, 'Log')